In [1]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup as bs

In [2]:
template = "https://www.indeed.com/jobs?q={}&l={}" 

In [3]:
def get_url(position,location):
#     generate url from position and location
    template = "https://www.indeed.com/jobs?q={}&l={}"
    url = template.format(position,location)
    return url

In [4]:
url = get_url("Senior Accountant","Charlotte, NC")

Extract raw html

In [5]:
response = requests.get(url)

In [6]:
response

<Response [200]>

In [7]:
response.reason

''

In [8]:
soup = bs(response.text,"html.parser")

In [9]:
cards = soup.find_all("div","jobsearch-SerpJobCard")

In [10]:
len(cards)

10

In [11]:
card = cards[0]

In [12]:
atag = card.h2.a

In [13]:
job_title = atag.get("title")

In [14]:
job_url = 'https://www.indeed.com'+atag.get('href')

In [15]:
company = card.find('span','company').text.strip()

In [16]:
job_location = card.find('div','recJobLoc').get('data-rc-loc')

In [17]:
job_summary = card.find('div','summary').text.strip()

In [18]:
post_date = card.find('span','date').text

In [19]:
today = datetime.today().strftime('%y-%m-%d')

In [20]:
try:
    job_salary = card.find('span','salaryText').text.strip()
except AttributeError:
    job_salary=""

Generalized the model with a function

In [21]:
def get_record(card):
    #extract job data from a single rcord
    atag = card.h2.a
    job_title = atag.get("title")
    job_url = 'https://www.indeed.com'+atag.get('href')
    company = card.find('span','company').text.strip()
    job_location = card.find('div','recJobLoc').get('data-rc-loc')
    job_summary = card.find('div','summary').text.strip()
    post_date = card.find('span','date').text
    today = datetime.today().strftime('%y-%m-%d')
    try:
        job_salary = card.find('span','salaryText').text.strip()
    except AttributeError:
        job_salary=""
    record = (job_title,job_url,company,job_location,job_summary,post_date,today,job_salary)
    return record

In [22]:
records = []
for card in cards:
    record = get_record(card)
    records.append(record)

In [23]:
while True:
    try:
        url = 'https://www.indeed.com'+soup.find('a',{'aria-label':'Next'}).get('href')
    except AttributeError:
        break
        
    response = requests.get(url)
    soup = bs(response.text,"html.parser")
    cards = soup.find_all("div","jobsearch-SerpJobCard")
    
    for card in cards:
        record = get_record(card)
        records.append(record)

In [24]:
len(records)

125

In [26]:
for r in records:
        print()
        print(i,r)

NameError: name 'i' is not defined

putting it all together

In [40]:
def get_url(position,location):
#     generate url from position and location
    template = "https://www.indeed.com/jobs?q={}&l={}"
    url = template.format(position,location)
    return url
def get_record(card):
    #extract job data from a single rcord
    atag = card.h2.a
    job_title = atag.get("title")
    job_url = 'https://www.indeed.com'+atag.get('href')
    company = card.find('span','company').text.strip()
    job_location = card.find('div','recJobLoc').get('data-rc-loc')
    job_summary = card.find('div','summary').text.strip()
    post_date = card.find('span','date').text
    today = datetime.today().strftime('%y-%m-%d')
    try:
        job_salary = card.find('span','salaryText').text.strip()
    except AttributeError:
        job_salary=""
    record = (job_title,job_url,company,job_location,job_summary,post_date,today,job_salary)
    return record

def main(location,position):
#     run the main program routine
    records = []
    url = get_url(position,location)
    #extract job data
    while True:    
        response = requests.get(url)
        soup = bs(response.text,"html.parser")
        cards = soup.find_all("div","jobsearch-SerpJobCard")

        for card in cards:
            record = get_record(card)
            records.append(record)
        try:
            url = 'https://www.indeed.com'+soup.find('a',{'aria-label':'Next'}).get('href')
        except AttributeError:
            break
        

In [41]:
main('Senior Accountants','Charlotte, NC')

In [80]:
import pandas as pd


In [81]:
df = pd.DataFrame(data=records)

In [85]:
df.columns =['JobTitle', 'JobUrl', 'Company', 'Location','Job Summary','Date Posted','Date Extracted','Salary']

In [91]:
df.head()

,JobTitle,JobUrl,Company,Location,Job Summary,Date Posted,Date Extracted,Salary
0,Senior Accountant,https://www.indeed.com/rc/clk?jk=e8934e1168d9c...,Childress Klein,"Charlotte, NC",This positionwill partner with team members an...,30+ days ago,21-03-16,
1,SENIOR ACCOUNTANT,https://www.indeed.com/rc/clk?jk=62cb75daa9592...,CABARRUS COUNTY,"Cabarrus County, NC","Prepares various reports, audit work papers, e...",30+ days ago,21-03-16,"$57,616 - $65,666 a year"
2,"Senior Accountant - Charlotte, NC",https://www.indeed.com/rc/clk?jk=85b3e39701b91...,Dentsply Sirona,"Charlotte, NC","Dentsply Sirona’s products provide innovative,...",14 days ago,21-03-16,
3,Senior Accountant - Global Services,https://www.indeed.com/rc/clk?jk=269c2fd02e8ed...,CliftonLarsonAllen,"Charlotte, NC",3+ years relative work experience as a staff o...,21 days ago,21-03-16,
4,Senior PE Fund Accountant and Client Service M...,https://www.indeed.com/rc/clk?jk=cfb16cac95ee8...,Apex Fund Services Limited,"Charlotte, NC",We are looking for highly motivated individual...,30+ days ago,21-03-16,


SOLUTION FROM ISREAL DRYER

In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
from time import sleep
from random import randint
from datetime import datetime

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47'
}

def get_url(position, location):
        template = 'https://www.indeed.com/jobs?q={}&l={}'
        url = template.format(position, location)
        return url

def get_record(card):
    '''Extract job date from a single record '''
    atag = card.h2.a
    try:
        job_title = atag.get('title')
    except AttributeError:
        job_title = ''
    try:
        company = card.find('span', 'company').text.strip()
    except AttributeError:
        company = ''
    try:
        location = card.find('div', 'recJobLoc').get('data-rc-loc')
    except AttributeError:
        location = ''
    try:
        job_summary = card.find('div', 'summary').text.strip()
    except AttributeError:
        job_summary = ''
    try:
        post_date = card.find('span', 'date').text.strip()
    except AttributeError:
        post_date = ''
    try:
        salary = card.find('span', 'salarytext').text.strip()
    except AttributeError:
        salary = ''
    
    extract_date = datetime.today().strftime('%Y-%m-%d')
    job_url = 'https://www.indeed.com' + atag.get('href')
    
    return (job_title, company, location, job_summary, salary, post_date, extract_date, job_url)

def main(position, location):
    # Run the main program reouting
    records = []  # creating the record list
    url = get_url(position, location)  # create the url while passing in the position and location.
    
    while True:
        print(url)
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')

        for card in cards:
            record = get_record(card)
            records.append(record)

        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
            delay = randint(1, 10)
            sleep(delay)
        except AttributeError:
            break

    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Job Title', 'Company', 'Location', 'Salary', 'Posting Date', 'Extract Date', 'Summary', 'Job Url'])
        writer.writerows(records)

In [ ]:
acccountants  = main('senior accountant', 'charlotte nc')